# Dataset from NER dataset

For our task we need to collect a dataset with following properties:
1. Sentence contains just one named entity
    + of type ORG for simplicity

In [9]:
from pathlib import Path
import re
import pymongo
from tqdm import tqdm

In [12]:
client = pymongo.MongoClient()
database = client["texts"]
collection = database["news"]
projection = {"text": 1}
texts = collection.find({}, projection)
total_texts = collection.estimated_document_count()
total_texts

538362

In [3]:
news_path = Path("./data/texts.json/")

In [4]:
import spacy

In [17]:
nlp = spacy.load("ru_core_news_lg")

In [14]:
from natasha import (Segmenter, Doc)
segmenter = Segmenter()

In [ ]:
data = []
for _doc in tqdm(texts, total=total_texts):
    text = _doc["text"]
    natashadoc = Doc(text)
    natashadoc.segment(segmenter)
    for idx, sentence in enumerate(natashadoc.sents):
        doc = nlp(sentence.text)
        org_count = 0
        ent = None
        for span in doc.ents:
            if span.label_ == "ORG":
                org_count += 1
                ent = span
        if org_count == 1:
            data.append({
                "text": sentence,
                "ent": span.text,
                "span": [span.start_char, span.end_char]
            })
            
            break
    
        
        
print("Suffice: ", len(data))

 20%|██████████████████████▍                                                                                           | 106199/538362 [48:00<3:11:39, 37.58it/s]

In [ ]:
with news_path.open("w") as fout:
    json.dump(data, fout, ensure_ascii=False)